In [1]:
import os
import numpy as np
import tifffile as tiff
import pandas as pd
import matplotlib.pyplot as plt
import cc3d
from skimage.measure import regionprops
from joblib import Parallel, delayed

def load_data_from_dir(input_dir):
    """Load 3D instance segmentation from a directory of 2D TIFF slices."""
    file_list = sorted([f for f in os.listdir(input_dir) if f.endswith('.tiff')])
    if not file_list:
        raise ValueError(f"No TIFF images found in {input_dir}")
    return np.stack([tiff.imread(os.path.join(input_dir, f)) for f in file_list])

def compute_psd(segmentation):
    """Compute volume, surface area, and diameter for each particle in the 3D segmentation."""
    labels = cc3d.connected_components(segmentation)  # Fast 3D labeling
    props = regionprops(labels)

    def compute_metrics(prop):
        if prop.label == 0:  # Exclude background
            return None

        instance_label = prop.label  # Instance ID
        volume = prop.area  # Voxel count as volume
        bbox = prop.bbox
        min_x, min_y, min_z, max_x, max_y, max_z = bbox
        surface_area = (
            np.sum(segmentation[min_x+1:max_x, min_y:max_y, min_z:max_z] != 
                   segmentation[min_x:max_x-1, min_y:max_y, min_z:max_z]) +
            np.sum(segmentation[min_x:max_x, min_y+1:max_y, min_z:max_z] != 
                   segmentation[min_x:max_x, min_y:max_y-1, min_z:max_z]) +
            np.sum(segmentation[min_x:max_x, min_y:max_y, min_z+1:max_z] != 
                   segmentation[min_x:max_x, min_y:max_y, min_z:max_z-1])
        )
        diameter = (6 * volume / np.pi) ** (1/3)  # Spherical equivalent diameter
        return instance_label, volume, surface_area, diameter

    results = Parallel(n_jobs=-1)(delayed(compute_metrics)(prop) for prop in props)
    results = [r for r in results if r is not None]  # Remove None values
    instance_labels, volumes, surface_areas, diameters = zip(*results) if results else ([], [], [], [])

    return np.array(instance_labels), np.array(volumes), np.array(surface_areas), np.array(diameters)

def remove_outliers(data):
    """Remove outliers using the IQR method and return both filtered data and kept indices."""
    if len(data) == 0:
        return data, np.arange(len(data))  # Return empty array and indices

    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 5.0 * IQR
    upper_bound = Q3 + 5.0 * IQR

    mask = (data >= lower_bound) & (data <= upper_bound)  # Boolean mask for non-outliers
    return data[mask], np.where(mask)[0]  # Return filtered data and valid indices

def bin_data(data, bin_edges):
    """Bin the data according to the bin edges and return the bin counts."""
    counts, _ = np.histogram(data, bins=bin_edges)
    return counts

def save_histogram(data, bin_edges, title, xlabel, save_path):
    """Save histogram plot with scaled x-axis, excluding zero values."""
    data = data[data > 0]  # Exclude zero values
    if len(data) == 0:
        print(f"Warning: No valid data for histogram {title}. Skipping.")
        return

    # num_bins = min(100, max(10, len(np.unique(data)) // 5))  # Adaptive bin count
    # bin_edges = np.linspace(data.min(), data.max(), num_bins)  # Scale x-axis

    plt.figure(figsize=(8, 6))
    plt.hist(data, bins=bin_edges, edgecolor='black', alpha=0.7)
    plt.xlabel(xlabel)
    plt.ylabel("Particle Count")
    plt.title(title)
    plt.grid(True)
    plt.savefig(save_path, dpi=300)
    plt.close()

def save_binned_psd(volumes, surface_areas, diameters, save_path, num_bins=50):
    """Save binned Particle Size Distribution (PSD) to CSV."""
    # Calculate the range for each dataset and round accordingly
    volume_min = np.floor(min(volumes))  # Round down
    volume_max = np.ceil(max(volumes))   # Round up
    surface_area_min = np.floor(min(surface_areas))  # Round down
    surface_area_max = np.ceil(max(surface_areas))   # Round up
    diameter_min = np.floor(min(diameters))  # Round down
    diameter_max = np.ceil(max(diameters))   # Round up

    # Check if the range is less than the number of bins, if so use unique values
    def create_bins(min_val, max_val, num_bins):
        bin_range = max_val - min_val
        if bin_range >= num_bins:
            return np.linspace(min_val, max_val, num_bins + 1, dtype=int)
        else:
            # Create extra bins by adjusting the bin width based on the range
            extra_bins = num_bins - bin_range  # Add extra bins if range is small
            return np.linspace(min_val, max_val + extra_bins, num_bins + 1, dtype=int)

    volume_bins = create_bins(volume_min, volume_max, num_bins)
    surface_area_bins = create_bins(surface_area_min, surface_area_max, num_bins)
    diameter_bins = create_bins(diameter_min, diameter_max, num_bins)

    # # Check the length of each bin array
    # print(f"Number of volume bins: {len(volume_bins)}")
    # print(volume_bins)
    # print(f"Number of surface area bins: {len(surface_area_bins)}")
    # print(surface_area_bins)
    # print(f"Number of diameter bins: {len(diameter_bins)}")
    # print(diameter_bins)

    # Bin the data
    volume_counts = bin_data(volumes, volume_bins)
    surface_area_counts = bin_data(surface_areas, surface_area_bins)
    diameter_counts = bin_data(diameters, diameter_bins)

    # Add ".0" after each integer in the bin range
    volume_bin_ranges = [f"{v1}.0-{v2}.0" for v1, v2 in zip(volume_bins[:-1], volume_bins[1:])]
    surface_area_bin_ranges = [f"{s1}.0-{s2}.0" for s1, s2 in zip(surface_area_bins[:-1], surface_area_bins[1:])]
    diameter_bin_ranges = [f"{d1}.0-{d2}.0" for d1, d2 in zip(diameter_bins[:-1], diameter_bins[1:])]

    # Create the DataFrame with valid bins
    binned_df = pd.DataFrame({
        "Volume Bin Range": volume_bin_ranges,
        "Volume Count": volume_counts,
        "Surface Area Bin Range": surface_area_bin_ranges,
        "Surface Area Count": surface_area_counts,
        "Diameter Bin Range": diameter_bin_ranges,
        "Diameter Count": diameter_counts
    })

    # Save to CSV
    binned_df.to_csv(save_path, index=False)
    print(f"Saved binned PSD results to {save_path}")

    return volume_bins, surface_area_bins, diameter_bins

def analyze_3d_instance_segmentation(input_dir, run_tag, name, save_dir, save):
    """Analyze 3D instance segmentation and compute particle size distribution (PSD)."""
    segmentation = load_data_from_dir(input_dir)
    instance_labels, volumes, surface_areas, diameters = compute_psd(segmentation)

    if save:
        csv_folder = os.path.join(save_dir, "csv", run_tag)
        hist_folder = os.path.join(save_dir, "hist", run_tag, name)
        os.makedirs(csv_folder, exist_ok=True)
        os.makedirs(hist_folder, exist_ok=True)

        # Save original CSV
        csv_path = os.path.join(csv_folder, f"{name}_raw_psd.csv")
        df = pd.DataFrame({
            "Instance": instance_labels,
            "Volume": volumes,
            "Surface Area": surface_areas,
            "Diameter": diameters
        })
        df = df[df["Volume"] > 0]  # Ensure zero values are excluded
        df.to_csv(csv_path, index=False)

        # Save filtered CSV (outliers removed)
        filtered_volumes, valid_indices = remove_outliers(volumes)
        filtered_surface_areas = surface_areas[valid_indices]
        filtered_diameters = diameters[valid_indices]
        filtered_instance_labels = instance_labels[valid_indices]  # Ensure labels match filtered data

        filtered_csv_path = os.path.join(csv_folder, f"{name}_filtered_psd.csv")
        filtered_df = pd.DataFrame({
            "Instance": filtered_instance_labels,
            "Volume": filtered_volumes,
            "Surface Area": filtered_surface_areas,
            "Diameter": filtered_diameters
        })
        filtered_df.to_csv(filtered_csv_path, index=False)

        # Save binned PSD CSVs
        raw_volume_bins, raw_surface_area_bins, raw_diameter_bins = save_binned_psd(volumes, surface_areas, diameters, os.path.join(csv_folder, f"{name}_raw_binned_psd.csv"), num_bins=50)
        filtered_volume_bins, filtered_surface_area_bins, filtered_diameter_bins = save_binned_psd(filtered_volumes, filtered_surface_areas, filtered_diameters, os.path.join(csv_folder, f"{name}_filtered_binned_psd.csv"), num_bins=25)

        # Save histograms
        save_histogram(volumes, raw_volume_bins, "Particle Volume Distribution", "Volume (voxels)", os.path.join(hist_folder, f"{name}_raw_volume_hist.png"))
        save_histogram(surface_areas, raw_surface_area_bins, "Particle Surface Area Distribution", "Surface Area (pixels)", os.path.join(hist_folder, f"{name}_raw_surface_hist.png"))
        save_histogram(diameters, raw_diameter_bins, "Particle Diameter Distribution", "Diameter (voxels)", os.path.join(hist_folder, f"{name}_raw_diameter_hist.png"))

        # Save filtered histograms
        save_histogram(filtered_volumes, filtered_volume_bins, "Filtered Particle Volume Distribution", "Volume (voxels)", os.path.join(hist_folder, f"{name}_filtered_volume_hist.png"))
        save_histogram(filtered_surface_areas, filtered_surface_area_bins, "Filtered Particle Surface Area Distribution", "Surface Area (pixels)", os.path.join(hist_folder, f"{name}_filtered_surface_hist.png"))
        save_histogram(filtered_diameters, filtered_diameter_bins, "Filtered Particle Diameter Distribution", "Diameter (voxels)", os.path.join(hist_folder, f"{name}_filtered_diameter_hist.png"))

        print(f"Saved results for {name} with outlier filtering.")


In [2]:
def setup_paths(dir_location, run_tag, output_to_cloud=False):
    if dir_location.lower() == 'internal':
        base_path = r'C:\Senior_Design'
    elif dir_location.lower() == 'external':
        base_path = r'D:\Senior_Design'
    elif dir_location.lower() == 'cloud':
        base_path = r'C:\Users\dchen\OneDrive - University of Connecticut\Courses\Year 4\Fall 2024\BME 4900 and 4910W (Kumavor)\Python\Files'
    elif dir_location.lower() == 'refine':
        base_path = r'D:\Darren\Files'
    else:
        raise ValueError('Invalid directory location type')

    base_input_path = os.path.join(base_path, 'outputs', 'tiff')
    psd_path = os.path.join(base_path, 'outputs', 'metrics', 'particle_size_dist')
    
    if output_to_cloud:
        psd_path = os.path.join(r'C:\Users\dchen\OneDrive - University of Connecticut\Courses\Year 4\Fall 2024\BME 4900 and 4910W (Kumavor)\Python\Files', 'outputs', 'metrics', 'particle_size_dist')

    input_tiff_path = os.path.join(base_input_path, run_tag)

    print('Paths set')
    print('Input:', input_tiff_path)
    print('Output:', psd_path)
    return input_tiff_path, psd_path

In [3]:
dir_location = 'refine'
run_tag = 'pretrained_tab40_gen35_clar35_foldsALL'
input_tiff_path, psd_path = setup_paths(dir_location, run_tag)
save = True  # Set to True if you want to save both histograms and PSD data
names = ['2_Tablet', '4_GenericD12', '5_ClaritinD12']

# # For ground truth instance segmentation
# input_tiff_path = r'C:\Users\dchen\OneDrive - University of Connecticut\Courses\Year 4\Fall 2024\BME 4900 and 4910W (Kumavor)\Python\Files\database\tablet_dataset\instance\tiff'
# run_tag = 'ground_truth'

for name in names:
    analyze_3d_instance_segmentation(os.path.join(input_tiff_path, name), run_tag, name, psd_path, save)

Paths set
Input: D:\Darren\Files\outputs\tiff\pretrained_tab40_gen35_clar35_foldsALL
Output: D:\Darren\Files\outputs\metrics\particle_size_dist
Saved binned PSD results to D:\Darren\Files\outputs\metrics\particle_size_dist\csv\pretrained_tab40_gen35_clar35_foldsALL\2_Tablet_raw_binned_psd.csv
Saved binned PSD results to D:\Darren\Files\outputs\metrics\particle_size_dist\csv\pretrained_tab40_gen35_clar35_foldsALL\2_Tablet_filtered_binned_psd.csv
Saved results for 2_Tablet with outlier filtering.
Saved binned PSD results to D:\Darren\Files\outputs\metrics\particle_size_dist\csv\pretrained_tab40_gen35_clar35_foldsALL\4_GenericD12_raw_binned_psd.csv
Saved binned PSD results to D:\Darren\Files\outputs\metrics\particle_size_dist\csv\pretrained_tab40_gen35_clar35_foldsALL\4_GenericD12_filtered_binned_psd.csv
Saved results for 4_GenericD12 with outlier filtering.
Saved binned PSD results to D:\Darren\Files\outputs\metrics\particle_size_dist\csv\pretrained_tab40_gen35_clar35_foldsALL\5_Claritin